# Concevez une application au service de la santé publique

Objectifs: 
-  Traiter le jeu de données afin de repérer des variables pertinentes pour les traitements à venir. Automatiser ces traitements pour éviter de répéter ces opérations. Le programme doit fonctionner si la base de données est légèrement modifiée (ajout d’entrées, par exemple). 
- Tout au long de l’analyse, produire des visualisations afin de mieux comprendre les données. Effectuer une analyse univariée pour chaque variable intéressante, afin de synthétiser son comportement. L’appel à projets spécifie que l’analyse doit être simple à comprendre pour un public néophyte. Soyez donc attentif à la lisibilité : taille des textes, choix des couleurs, netteté suffisante, et variez les graphiques (boxplots, histogrammes, diagrammes circulaires, nuages de points…) pour illustrer au mieux votre propos.
-  Confirmer ou infirmer les hypothèses  à l’aide d’une analyse multivariée. Effectuer les tests statistiques appropriés pour vérifier la significativité des résultats.
- Élaborer une idée d’application. Identifier des arguments justifiant la faisabilité (ou non) de l’application à partir des données Open Food Facts.
- Rédiger un rapport d’exploration et pitcher votre idée durant la soutenance du projet.

# Sommaire
### [I Exploration du jeu de données et sélection des variables](#I-Exploration-du-jeu-de-données-et-sélection-des-variables)
__[I.1 Chargement des données](#I.1-Chargement-des-données)__

__[I.2 Sélection des variables](#I.2-Sélection-des-variables)__\
[I.2.1 product_name et generic_name](#I.2.1-product_name-et-generic_name)\
[I.2.2 quantity - brands_tags](#I.2.2-quantity---brands_tags)\
[I.2.3 Categories](#I.2.3-Categories)\
[I.2.4 Origins - Manufacturing_places - purchase places - countries](#I.2.4-Origins---Manufacturing_places---purchase-places---countries)\
[I.2.5 labels - stores](#I.2.5-labels---stores)\
[I.2.6 PNNS groups](#I.2.6-PNNS-groups)\
[I.2.7 Variables en lien avec les données nutritionnelles](#I.2.7-Variables-en-lien-avec-les-données-nutritionnelles)\
[I.2.8 Conclusion](#I.2.8-Conclusion)

__[Création des dataframes pour chaque idée d'application](#Création-des-dataframes-pour-chaque-idée-d'application)__\
[Première idée](#Première-idée)\
[Deuxième idée](#Deuxième-idée)\
[Troisième idée](#Troisième-idée)
### [II Nettoyage des datasets](#II-Nettoyage-des-datasets)
__[II.1 Chargement des datasets](#II.1-Chargement-des-datasets)__

__[II.2 Nettoyage du jeu de données: valeurs incohérentes et/ou extrêmes](#II.2-Nettoyage-du-jeu-de-données:-valeurs-incohérentes-et/ou-extrêmes)__\
[II.2.1 Valeurs typiques et atypiques des compositions énergétiques totale et par macro-nutriment](#II.2.1-Valeurs-typiques-et-atypiques-des-compositions-énergétiques-totale-et-par-macro-nutriment)\
[II.2.2 Valeurs typiques et atypiques des compositions énergétiques totales par groupe d'aliments (pnns_groups)](#II.2.2-Valeurs-typiques-et-atypiques-des-compositions-énergétiques-totales-par-groupe-d'aliments-(pnns_groups))\
[II.2.3 Traitement des valeurs atypiques: cohérence des variables](#II.2.3-Traitement-des-valeurs-atypiques:-cohérence-des-variables)\
[II.2.3.i Traitement sur la sous-catégorie des carottes râpées](#II.2.3.i-Traitement-sur-la-sous-catégorie-des-carottes-râpées)\
[II.2.3.ii Traitement sur les données générales](#II.2.3.ii-Traitement-sur-les-données-générales)\
[II.2.4 Fusion des catégories Fruits et fruits](#II.2.4-Fusion-des-catégories-Fruits-et-fruits)

__[II.3 Traitement des valeurs manquantes](#II.3-Traitement-des-valeurs-manquantes)__\
[II.3.1 Remplissage de fiber_kcal](#II.3.1-Remplissage-de-fiber_kcal)\
[II.3.2 Cas des autres macronutriments: fat, carbohydrates et proteins](#II.3.2-Cas-des-autres-macronutriments:-fat,-carbohydrates-et-proteins)\
[II.3.3 Nettoyage des valeurs aberrantes (bis)](#II.3.3-Nettoyage-des-valeurs-aberrantes-(bis))
### [III Automatisation des traitements](#III-Automatisation-des-traitements)\

# I Exploration du jeu de données et sélection des variables

## I.1 Chargement des données et sélection des variables

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import seaborn as sns
from matplotlib.ticker import AutoMinorLocator
import missingno as msno
from functions import *
from cleaning import *

In [ ]:
# dataset à analyser
dataset = "./en.openfoodfacts.org.products.csv"

# variables retenues pour l'analyse
selected_cols = ['product_name',
                 'categories_tags',
                 'countries_tags',
                 'pnns_groups_2',
                 'energy-kcal_100g',
                 'fat_100g',
                 'saturated-fat_100g',
                 'nutrition-score-fr_100g',
                 'nutriscore_grade',
                 'ecoscore_score_fr',
                 'ecoscore_grade_fr',
                 'fiber_100g',
                 'proteins_100g',
                 'carbohydrates_100g']

# restriction aux pays pour lesquels countries_tags="en:france"
restr="en:france"

# Chargement du dataset 
df1 = load_select(dataset, selected_cols, restr)

# Restriction aux "One dish meal", "Fruits" et "Yoghourts"
selected_groups = ['One-dish meals','Fruits', 'Milk and yogurt', 'fruits']
df1 = df1.loc[df_app1.pnns_groups_2.isin(selected_groups)]
    
###### fusion des pnns_groups "Fruits" et "fruits"
df1.loc[:, "pnns_groups_2"] = df_app1["pnns_groups_2"].apply(lambda x: "Fruits" if x=='fruits' else x)

##### Ecriture du dataframe dans un fichier csv
df1.to_csv('./df1.csv', index_label=False)

## II.2 Nettoyage du jeu de données: valeurs incohérentes et/ou extrêmes

### II.2.1 Conversion des valeurs en grammes en valeurs en kcal

Comme notre application consiste en la sélection de produits permettant d'atteindre les ANC, et que ces dernières sont généralement données en % apports énergétiques, on commence par créer de nouvelles colonnes contenant les valeurs en kcal associées aux macronutriments, et renseignées en g.

In [2]:
# Colonnes sur lesquelles opérer les conversions
start_cols = ["fat_100g", "carbohydrates_100g", "proteins_100g", "fiber_100g"]

# Colonnes à créer avec le coefficient de conversion g --> kcal associé
new_cols = [("fat_kcal", 9), ("carbohydrates_kcal", 4), ("proteins_kcal", 4), ("fiber_kcal", 1.9)]

# Conversions et créations des colonnes
for (nc, sc) in zip(new_cols, start_cols):
    df1 = weight_to_energy(df1, sc, nc[0], nc[1])

    
# Calcul de la somme des valeurs obtenues pour chaque ligne et création d'une nouvelle colonne
# "total_energy_from_nutriments"
df1.loc[:, "total_energy_from_nutriments"] = df1[["fat_kcal","carbohydrates_kcal","proteins_kcal", "fiber_kcal"]].apply(lambda x: np.sum(x), axis=1)

NameError: name 'df_app1' is not defined

### II.2.2 Nettoyage des valeurs aberrantes (1): conversion en kcal des valeurs données en kJ

Certaines valeurs renseignées dans la colonne "energy-kcal_100g" apparaissent être en fait des valeurs données en kJ. On récupère ces valeurs en sélectionnant les valeurs dans un intervalle ad hoc autour de 4,18 $\times$ total_energy_from_nutriments, et en leur appliquant le facteur de conversion $0,239=\frac{1}{4,18}$.

In [ ]:
## Calcul du ratio de l'énergie calculée à partir des valeurs de macronutriments sur l'énergie renseignée
#calculated_e = df.loc[df_app1["energy-kcal_100g"].notna(), "total_energy_from_nutriments"]
#actual_e = df.loc[df_app1["energy-kcal_100g"].notna(), "energy-kcal_100g"]
#ratio  = calculated_e/actual_e
col2="energy-kcal_100g"
col1="total_energy_from_nutriments"
ratio = create_distrib(df1, col1=col1, col2=col2, reverse_cols=False, method="ratio", mask="notna")

# on ne garde dans la Series ratio que les valeurs comprises entre 0.1 et 0.4. Les valeurs intéressantes 
# se trouvent vers 0.25 et ainsi on peut calculer une moyenne et un écart-type raisonnables sur cette 
# distribution
ratio = ratio[(ratio>0.1) & (ratio<0.4)]
#sub_ratio = sub_ratio[sub_ratio<0.4]

# Moyenne et écart-type de la "distribution" ratio
mean = ratio.mean()
sigma = ratio.std()

# cut-off associés
cut_off_high = mean + sigma
cut_off_low = mean - sigma

df1 = joule_to_kcal(df1, "energy-kcal_100g", "total_energy_from_nutriments", cut_off_high, cut_off_low)

### II.2.3  Nettoyage des valeurs aberrantes (2): filtrage par la différence entre énergie calculée et énergie renseignée

Certaines valeurs sont simplement aberrantes: l'énergie globale renseignée ne correspond pas du tout à l'énergie qu'on peut calculer à partir des macronutriments. En repérant ces valeurs, on peut ainsi retirer un bon nombre de valeurs incohérentes.

In [ ]:
# Remplacement des valeurs nulles par des NaN pour faciliter le traitement
for col in ["total_energy_from_nutriments", "energy-kcal_100g"]:
    df1 = replace_with_nan(df1, col)

# On arrondit les valeurs de total_energy_from_nutriments à l'entier inférieur, suivant ainsi ce qui semble
# être la pratique courante.
rounded_val = df1["total_energy_from_nutriments"].apply(lambda x: np.floor(x))
df1.loc[:, "total_energy_from_nutriments"] = rounded_val

# Création de la colonne complete_vars: True si toutes les colonnes target_cols sont renseignées, False sinon.
target_cols = ["total_energy_from_nutriments",
          "energy-kcal_100g",
          "fat_kcal",
          "carbohydrates_kcal",
          "proteins_kcal",
          "fiber_kcal"]

df1 = make_mask(df1, target_cols, "isna")

# On crée la distribution représentant la différence total_energy_from_nutriments - energy-kcal_100g
#calculated_e = df_app1.loc[df_app1.e_nn==True, "total_energy_from_nutriments"]
#actual_e = df_app1.loc[df_app1.e_nn==True, "energy-kcal_100g"]
#diff = pd.Series(actual_e-calculated_e, name="diff")
#col1="energy-kcal_100g"
#col2="total_energy_from_nutriments"
diff = create_distrib(df1, mask="e_nn")
# Ajout d'une colonne contenant la valeur de la différence entre l'énergie calculée  et l'énergie renseignée.
df1.loc[:, "diff"] = diff

# Mise à jour du dataframe
Y = df_app1.loc[df_app1.e_nn==True].copy()
drop_index = Y.loc[np.abs(Y["diff"]-diff.mean())>diff.std()].index
df1.drop(index=drop_index, inplace=True)

# Création d'un dataframe ne contenant que les bonnes valeurs
df1_no = df1.copy()
df1_no = df1_no.loc[np.abs(df_app1_no["diff"]-diff.mean())<diff.std()]

## II.3 Traitement des valeurs manquantes

On peut essayer d'imputer les valeurs manquantes en remplaçant par les moyennes pour une même catégorie. La colonne "categories_tags" va nous servir à cela. Chaque valeur de categorie_tags est en fait une suite de tags. La structure de cette variable est telle que le premier tag est le plus général, et le dernier le plus particulier. En utilisant le dernier tag, on augmente donc nos chances d'imputer avec une valeur la moins "incorrecte" possible. On illustre cela en regardant comment les first tags et les last_tags sont peuplés.

In [ ]:
Y = df1.copy()
# 1. Imputation à l'aide des moyennes de groupe
for c in ["fat_kcal", "carbohydrates_kcal", 'proteins_kcal', 'fiber_kcal']:
    Y[c].fillna(round(Y.groupby(["pnns_groups_2", "last_tag"], dropna=False)[c].transform("mean"), 1)
                , inplace=True)
    
# 1. Mise à jour du dataframe df_app1 avec les valeurs renseignées.
#df_app1 = Y

# 2. On remplace les valeurs nulles de la colonne total_energy_from_nutriments pour faciliter le traitement
#df_app1 = replace_with_nan(df_app1, "total_energy_from_nutriments")

# 2. Calcul des valeurs total_energy_from_nutriments à partir des valeurs des macronutriments 
# nouvellement renseignées.
#Y = df_app1.copy()
Y["total_energy_from_nutriments"].fillna(Y["fat_kcal"]+Y["carbohydrates_kcal"]+Y["proteins_kcal"]+Y["fiber_kcal"], inplace=True)
Y["energy-kcal_100g"].fillna(round(Y["total_energy_from_nutriments"], 1), inplace=True)

# 2. Mise à jour du dataframe
df1 = Y

### II.3.2 Nettoyage des valeurs aberrantes (bis)

In [ ]:
# On crée la distribution représentant la différence entre total_energy... et energy_...
#calculated_e = df_app1.loc[:, "total_energy_from_nutriments"]
#actual_e = df_app1.loc[:, "energy-kcal_100g"]
#diff = pd.Series(actual_e-calculated_e, name="diff")
diff = create_distrib(df1, method="diff")

# Mise à jour de la colonne diff
df1.loc[:, "diff"] = diff

# Récupération des index à retirer du dataframe
drop_index = df1.loc[np.abs(df1["diff"]-diff.mean())>diff.quantile(0.95)].index # utilisation du 95ème percentile 
                                                                            # plutôt que l'écart-type, trop 
                                                                            # large à cause des outliers
# On enlève les lignes correspondant aux outliers.
df1.drop(index=drop_index, inplace=True)

## II.4 Traitement du nutriscore_grade

Nous allons regarder les deux variables nutrition-score-fr_100g ainsi que nutriscore_grade de plus près, en gardant en tête que le nutriscore_grade est défini de la façon suivante:
- "A": score entre -15 et -2
- "B": score entre -1 et +3
- "C": score entre +4 et +11
- "D": score entre +12 et +16
- "E": score supérieur ou égal à 17

In [ ]:
df1["nutriscore_grade"] = df1["nutrition-score-fr_100g"].apply(lambda x: assign_grade(x))